# Setup



## Colab setup
---------- Start of generic setup code to get kaggle credentials login and data to work with. Copied much from Workshop 2 ----------

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
!pip install -U -q scikeras
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [2]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

https://drive.google.com/file/d/1DeYgEzZ8Bs8Y3QsHd5YsV-3D-SzVAm4x

In [3]:
# GET data from file I've uploaded to my drive. Use your own kaggle.json and google auth logins etc
downloaded = drive.CreateFile({"id": "1DeYgEzZ8Bs8Y3QsHd5YsV-3D-SzVAm4x"})
downloaded.GetContentFile("predict-energy-consumption.zip")

In [4]:
!unzip predict-energy-consumption.zip

!ls
!mkdir data
!mv *.csv data/

Archive:  predict-energy-consumption.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
predict-energy-consumption.zip	sample_submission.csv  train.csv
sample_data			test.csv


In [5]:
from google.colab import files

files.upload()
# upload your kaggle.json file

Saving kaggle.json to kaggle.json


Processing data from the source.

In [6]:
import json

!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [7]:
# GENERIC SUBMIT FUNCTION (uncomment when you want to submit)

# submission = pd.DataFrame(
#    {'key': test.key, 'fare_amount': val_predictions},
#    columns = ['key', 'fare_amount'])
# submission.to_csv('submission.csv', index = False)


#!kaggle competitions submit -c predict-electricity-consumption -f submission.csv -m "[TEAM NAME] Submission"

## Libraries setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Data exploration

## Preprocessing

### Modification functions

In [2]:
def replaceNanWithZero(test, train, labels):
    train_labels = labels.replace(np.nan, 0.0)

    for col in train.columns:
        train[col] = train[col].replace(np.nan, 0.0)

    for col in test.columns:
        test[col] = test[col].replace(np.nan, 0.0)

    return test, train, train_labels

In [3]:
# PCA to get a lesser number of features, then going to feed into basic nn see what we can do
from sklearn.decomposition import PCA


def PCA_Performance(train, test):
    variance = [0.5, 0.6, 0.75, 0.8, 0.85, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99]

    for var in variance:
        pca = PCA(n_components=var)
        pca.fit(train)
        transformed_Train = pca.transform(train)
        # transformed_Test = pca.transform(test)
        print(
            f"Retained {var} of variance with {len(transformed_Train.T)} number of features"
        )

### Data modification

In [4]:
# Here we can split into train/test for verification while we train whatever works)
train = pd.read_csv("data/train.csv")
train_labels = train["consumption"]  # y vals in this case are concumption

# GIVEN time is a string we just gonna drop that mfer, cbf processing it etc
train = train.drop(["time"], axis=1)
train = train.drop(["consumption"], 1)
# print(train)

test = pd.read_csv("data/test.csv")
test = test.drop(["time"], axis=1)
# Have to predict consumption for these values. Consumption unkown.

# print(train)
# print(test)

# At this point should have a train and test without Nan values:)

/var/folders/qn/lk9fg3fs0hg9v8b7zh5lbybw0000gn/T/ipykernel_60775/1851060134.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop(["consumption"], 1)


---------- End of generic setup code. Actual programming can begin after here ----------

In [5]:
# USE the code we wrote above
test_, train, labels = replaceNanWithZero(test, train, train_labels)

PCA_Performance(train, test)

Retained 0.5 of variance with 1 number of features
Retained 0.6 of variance with 1 number of features
Retained 0.75 of variance with 1 number of features
Retained 0.8 of variance with 1 number of features
Retained 0.85 of variance with 1 number of features
Retained 0.9 of variance with 2 number of features
Retained 0.95 of variance with 3 number of features
Retained 0.96 of variance with 4 number of features
Retained 0.97 of variance with 4 number of features
Retained 0.98 of variance with 5 number of features
Retained 0.99 of variance with 5 number of features
